# 3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo.



## Resumen
En este ejercicio nuestro dataset va a ser un conjunto de tweets de Donald Trump.


El objetivo de este ejercicio es analizar el tipo de mensajes que escribe en tweeter Donald Trump para poder imitarle creando nuestros propios tweets.

Este tipo de problemas entran dentro de lo que conocemos como Lenguaje Modeling, en concreto y para el caso que nos ocupa usaremos RNNLM (Recurrent Neural Network Language Modeling) explicado en los ultimos dias de clase

## Librerias
Tanto gensim como spacy vienen ya preinstaladas

In [0]:
import spacy
import numpy as np

import pickle
import json
import os
import csv
import pprint as pp

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import shuffle, choice, sample

from sklearn.model_selection import StratifiedShuffleSplit

from copy import copy

import warnings
warnings.filterwarnings('ignore')

#nlp = spacy.load('en_core_web_sm')

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab as pl
from IPython import display

sns.set(color_codes=True)

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.


## Funciones/Classes

In [0]:
# En la funcion preprocess pondremos diferentes opciones de preprocesado del texto
# e iremos probando con distintas combinaciones (comentando y descomentando) para ver
# cual nos genera un mejor modelo.
def preprocess(text, lema = True):

  # Eliminamos los espacios en blanco por delante y detras
  text = text.strip()

  #Tokenizamos
  doc = nlp(text)
  words = [t for t in doc]

  # Quitamos los signos de puntuacion
  #words = [t for t in doc if not t.is_punct]
  
  # Quitamos las stop_words
  words = [t for t in words if not t.is_stop]

  # Eliminamos palabras de tamaño menor de tres
  #words = [t for t in words if len(t) > 2 and  t.text.isalpha()]
   
  # Lematizamos
  if(lema):
    words = [t.lemma_ for t in words]
  else:
    words = [t.text for t in words]

  # Ponemos las palabras en minusculas
  words = [t.lower() for t in words]

  #Cambiamos las urls por <URL>
  words = ['<URL>' if "http" in word else word for word in words]

  # Por ultimo eliminamos los caracteres que no queremos que aparezcan
  all_printables_chars = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
  printables_chars = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ! ?'
  words = [''.join([c if c in printables_chars else '' for c in word]) for word in words]
  
  # Generamos el string de salida
  result = ' '.join(words)
  return result.strip()

In [0]:
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': len(vocab),
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2c[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))

In [0]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()
        

In [0]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [0]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
        
    def compile_model(self, params={}):

        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        embedding = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        
        lstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')
        lmlstm = Bidirectional(lstm)(embedding)
        
        stacklstm = CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stack')
        stackedlstm = stacklstm(lmlstm)

        out = Dense(params['vocab'], activation='softmax')(stackedlstm)

        model = Model(lm_inputs, out)
        model.compile(
            optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        model.summary()
        return model
        
        
    def train(self, model, data, params={}):
        callbacks = self._get_callbacks()

        if 'shuffle' in params and params['shuffle']:
            shuffle(data)

        sentences, next_chars = zip(*data)
        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))
        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = c2id[char] if char in c2id else c2id['<UNK>']
            y[i, c2id[next_chars[i]] if next_chars[i] in c2id else c2id['<UNK>']] = 1

        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)
        
    def predict(self, model, data, params={}):
        for diversity in [0.2, 0.6, 1.2]:
            print('------ diversity: ', diversity)
            sentence = copy(data)
            generated = copy(data)
            for i in range(len(data), 240): # si fuese un tweet
                x_pred = np.zeros((1, params['maxlen']))
                for t, char in enumerate(sentence):
                    x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                
                preds = self.model.predict(x_pred, verbose=0)[0]
                next_index = sample_pred(preds, diversity)
                next_char = id2c[next_index]

                if next_char == '<EOS>':
                    break
                
                generated += [next_char]
                sentence = sentence[1:]
                sentence+= [next_char]
            
            print(''.join(generated))
        
    def _get_callbacks(self, model_path='model_lm.h5'):
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, period=2)
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        st = Sampletest()
        hd = HistoryDisplay()
        return [st, rlr, es]

## Dataset

In [0]:
from google.colab import files
import json

In [10]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving realDonaldTrump.json to realDonaldTrump (1).json
User uploaded file "realDonaldTrump.json" with length 524785 bytes


In [11]:
json_file = uploaded['realDonaldTrump.json']
dataset = json.loads(json_file)
len(dataset)


2995

### Preproceso
Vamos a reutilizar la funcion de preproceso del ejercicio anterior con alguna modificación. por ejemplo cambiamos las urls que aparecen en los tweets por el token <URL>

In [0]:
# En esta ocasion usamos el modelo en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [0]:
#Extraemos el texto de los tweets del diccionario
documents = []
for key, value in dataset.items():
  #Aplicamos el preproceso al texto, en caso de no querer aplicar ningun preproceso bastara con comentar esta linea
  value = preprocess(value,False)
  documents.append(value)

In [16]:
documents[0:10]

['thoughts prayers families serviceman killed fellow servicemen wounded somalia  truly heroes',
 'thoughts prayers families serviceman killed fellow servicemen wounded somalia  truly heroes',
 'heading canada g7 talks center long time unfair trade practiced united states  singapore talks north korea denuclearization  wo nt talking russian witch hunt hoax !',
 'congratulations washington capitals great play winning stanley cup championship  alex ovechkin  team captain  spectacular  true superstar ! dc popping  ways  time !',
 'looking forward straightening unfair trade deals g7 countries  nt happen  come better !',
 'canada charges us 270    tariff dairy products ! nt tell  ? fair farmers !',
 'obama  schumer pelosi north korea  weak crime  high tax schumer telling summit dems set  schumer failed north korea iran  nt need advice !',
 'nt european union canada informing public years massive trade tariffs non  monetary trade barriers us totally unfair farmers  workers  amp  companies  tar

In [0]:
# Tokenizamos documents
tokenized = [list(x) for x in documents] # tokenizado por caracteres!!

In [0]:
#tokenized[0]

In [0]:
init_chars = [x[:5] for x in tokenized] # me quedo con los 4 caracteres iniciales

for i in range(len(init_chars)):
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>')
    init_chars[i] = tmp[:5]

### Algunos datos sobre el dataset

In [22]:
maxlen = max([len(x) for x in tokenized])
avglen = sum([len(x) for x in tokenized]) / len(tokenized)
maxlen, avglen

(251, 102.76861435726211)

In [24]:
total_tokens = [t for s in documents for t in s]
len(total_tokens)

307792

## Vocabulario

In [25]:
from collections import Counter
vocab_counter = Counter(total_tokens)
vocab = [w for w, v in vocab_counter.items() if v>2] # filtrado de caracteres que minimo hayan aparecido 2 veces
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab

nb_vocab = len(vocab)
nb_vocab

46

In [0]:
c2id = {k: i for i, k in enumerate(vocab)}
id2c = {i: k for k, i in c2id.items()}

In [27]:
maxlen = 5
step = 1
data_train = []

for x in tokenized: # x es una lista con mi frase "tokenizada"
    x.insert(0, '<SOS>')
    x.append('<EOS>')
    for i in range(0, len(x)-maxlen, step):
        data_train.append((x[i:i+maxlen], x[i+maxlen]))

len(data_train)

298807

In [0]:
# Variables globales
# Cada Cuanto escribimos una frase de ejemplo durante el entrenamiento
SAMPLE_EVERY = 4
PLOT_EVERY = 1 

## Compilar

In [0]:
# Hyperparametros
compile_params = {
    'maxlen': maxlen,
    'vocab': nb_vocab,
    'emb_feats': 100,
    'rnn_hidden': 256
}

In [30]:
lm = LM()
lm_model = lm.compile_model(params=compile_params)






Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5, 100)            4600      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stack (CuDNNLSTM)            (None, 256)               788480    
_________________________________________________________________
dense_1 (Dense)              (None, 46)                11822     
Total params: 1,538,086
Trainable params: 1,538,086
Non-trainable params: 0
_________________________________________________________________


## Entrenar

In [0]:
# Hyperparametros
train_params = {
    'epochs': 300,
    'batch_size': 512,
    'shuffle': True,
    'vocab': nb_vocab,
    'maxlen': maxlen,
}

In [32]:
lm.train(lm_model, data=data_train, params=train_params)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/300





298807/298807 [==============================] - 25s 84us/step - loss: 2.3842 - acc: 0.3018
Epoch 2/300
298807/298807 [==============================] - 10s 32us/step - loss: 1.8316 - acc: 0.4662
Epoch 3/300
298807/298807 [==============================] - 10s 32us/step - loss: 1.5636 - acc: 0.5469
Epoch 4/300
298807/298807 [==============================] - 10s 32us/step - loss: 1.4256 - acc: 0.5822
Epoch 5/300
298807/298807 [==============================] - 10s 32us/step - loss: 1.3402 - acc: 0.6029
----- diversity: 0.2
<SOS>yesterday  amp  desperation  amp  dems                                             URL URL
----- diversity: 0.6
<SOS>yesterday  d president control tell  north korea  !
----- diversity: 1.2
<SOS>yesterday  gattenewsmaker need ser bell
Epoch 6/300
298807/298807 [==============================] - 9s 32us/step - loss: 1.2811 - acc: 0.6174
Epoch 7/300
29

## Recopilacion de Frases


*   thank stuart votes  tariffs comey corrupt russia hoax control office making race real disgrace begins floods  commencemen  women carried secretary
*   foxnews political caputo saying block requiremendous united state histories cut let option young woman noon !

*  russia covered  president   phony russian collusion  west black texas  stock market hit  time  border  shoplifting  working hard  usa

*  foxnews going  heading hard blumenthal speak  home security  amp  democrats far great people state  military  big ratings china  comey  states  trump  authorization trade  good  american senate  paul mandate  la  ms13 gang member people   wall  build  business  history !

*  great book   amp  republicans want charter president trump  beautiful president obama  bill singapore use pass good mexico michael capable  great !

*  protection russia  comey  fake news  money spend terminate  agreement  page  proudly playing lost cnn

